# Inception-V1 (GoogLeNet) Training for Brain Tumor Classification

This notebook trains an Inception-V1 (GoogLeNet) model from scratch to classify brain MRI images into 4 categories:
- **NO_TUMOR**: Healthy brain (no tumor detected)
- **GLIOMA**: Glioma tumor type
- **MENINGIOMA**: Meningioma tumor type
- **PITUITARY**: Pituitary tumor type

## Overview

This notebook includes:
1. Data loading and preprocessing (using same pipeline as VGG16 preprocessing)
2. Inception-V1 (GoogLeNet) architecture implementation from scratch
3. Training pipeline with validation
4. Model saving based on best validation accuracy

## Requirements

Make sure you have installed all required packages:
```bash
pip install torch torchvision seaborn pandas scikit-learn matplotlib numpy tqdm
```

Or install from requirements.txt:
```bash
pip install -r requirements.txt
```


## 1. Initialization


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm
import os
import random
from pathlib import Path

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')


## 2. Configuration


In [ ]:
# Configuration
DATA_DIR = 'data/vgg16_classification'
MODEL_SAVE_PATH = 'inception_model.pth'

# Training hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
NUM_CLASSES = 4

# Class names
CLASS_NAMES = ['NO_TUMOR', 'GLIOMA', 'MENINGIOMA', 'PITUITARY']

print(f"Configuration:")
print(f"  Data directory: {DATA_DIR}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Number of epochs: {NUM_EPOCHS}")
print(f"  Number of classes: {NUM_CLASSES}")
print(f"  Model save path: {MODEL_SAVE_PATH}")


## 3. Custom Dataset Class (from preprocess_vgg16 pipeline)


In [ ]:
# Custom dataset class that uses CSV metadata to prevent data leakage
from torch.utils.data import Dataset
from PIL import Image

class FilteredImageFolder(Dataset):
    """
    Custom dataset that loads images based on CSV metadata to prevent data leakage.
    Only loads images whose original_filename is NOT in other splits.
    """
    def __init__(self, metadata_df, split_name, transform=None, base_dir='data/vgg16_classification'):
        """
        Args:
            metadata_df: DataFrame with columns: image_path, full_path, class, split, filename, original_filename (optional)
            split_name: 'train', 'val', or 'test'
            transform: Image transforms
            base_dir: Base directory for images
        """
        self.split_name = split_name
        self.transform = transform
        self.base_dir = base_dir
        
        # Filter by split
        split_df = metadata_df[metadata_df['split'] == split_name].copy()
        
        # For train: use augmented metadata, filter by original_filename not in test/val
        if split_name == 'train':
            # Get original filenames from test and val splits (from original metadata)
            if 'original_filename' in metadata_df.columns:
                # This is augmented metadata
                orig_metadata_path = 'data/dataset_metadata.csv'
                if os.path.exists(orig_metadata_path):
                    orig_df = pd.read_csv(orig_metadata_path)
                    test_originals = set(orig_df[orig_df['split'] == 'test']['filename'].unique())
                    val_originals = set(orig_df[orig_df['split'] == 'val']['filename'].unique())
                    excluded_originals = test_originals.union(val_originals)
                    # Filter: only keep images whose original_filename is NOT in test/val
                    split_df = split_df[~split_df['original_filename'].isin(excluded_originals)]
        
        # For val/test: use original metadata, ensure no overlap with train
        elif split_name in ['val', 'test']:
            # Get train original filenames (from augmented metadata if available)
            aug_metadata_path = 'data/augmented_dataset_metadata.csv'
            if os.path.exists(aug_metadata_path):
                aug_df = pd.read_csv(aug_metadata_path)
                train_originals = set(aug_df[aug_df['split'] == 'train']['original_filename'].unique())
                # Filter: only keep images whose filename is NOT in train
                split_df = split_df[~split_df['filename'].isin(train_originals)]
        
        self.samples = []
        self.classes = sorted(split_df['class'].unique().tolist())
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        for _, row in split_df.iterrows():
            # Use full_path if available, otherwise construct from base_dir and image_path
            if pd.notna(row.get('full_path')):
                img_path = row['full_path']
            else:
                img_path = os.path.join(base_dir, row['image_path'])
            
            # Normalize path separators
            img_path = img_path.replace('\\', '/')
            
            if os.path.exists(img_path):
                label = self.class_to_idx[row['class']]
                self.samples.append((img_path, label))
            else:
                print(f"Warning: Image not found: {img_path}")
        
        print(f"Loaded {len(self.samples)} images for {split_name} split (filtered from {len(split_df)} rows in CSV)")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            # Return a black image as fallback
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


## 4. Data Loading and Preprocessing


In [ ]:
# Data transforms (same as VGG16 preprocessing pipeline)
# Training: only normalization (augmentation already applied via augment_training_data.py)
# Use train_augmented directory which contains pre-augmented images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation/Test: only normalization (no augmentation)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets using CSV metadata to prevent data leakage
print("Loading datasets using CSV metadata files to prevent data leakage...")

# Load metadata CSV files
augmented_metadata_path = 'data/augmented_dataset_metadata.csv'
original_metadata_path = 'data/dataset_metadata.csv'

if os.path.exists(augmented_metadata_path):
    aug_metadata_df = pd.read_csv(augmented_metadata_path)
    print(f"Loaded augmented metadata: {len(aug_metadata_df)} rows")
else:
    print(f"Warning: {augmented_metadata_path} not found. Creating empty DataFrame.")
    aug_metadata_df = pd.DataFrame()

if os.path.exists(original_metadata_path):
    orig_metadata_df = pd.read_csv(original_metadata_path)
    print(f"Loaded original metadata: {len(orig_metadata_df)} rows")
else:
    print(f"Warning: {original_metadata_path} not found. Creating empty DataFrame.")
    orig_metadata_df = pd.DataFrame()

# Use FilteredImageFolder for train (from augmented metadata)
if len(aug_metadata_df) > 0:
    train_dataset = FilteredImageFolder(aug_metadata_df, 'train', transform=train_transform, base_dir=DATA_DIR)
else:
    print("Falling back to ImageFolder for train (no augmented metadata found)")
    train_dir = os.path.join(DATA_DIR, 'train_augmented')
    if not os.path.exists(train_dir):
        train_dir = os.path.join(DATA_DIR, 'train')
    train_dataset = ImageFolder(root=train_dir, transform=train_transform)

# Use FilteredImageFolder for val and test (from original metadata)
if len(orig_metadata_df) > 0:
    val_dataset = FilteredImageFolder(orig_metadata_df, 'val', transform=val_test_transform, base_dir=DATA_DIR)
    test_dataset = FilteredImageFolder(orig_metadata_df, 'test', transform=val_test_transform, base_dir=DATA_DIR)
else:
    print("Falling back to ImageFolder for val/test (no original metadata found)")
    val_dataset = ImageFolder(root=os.path.join(DATA_DIR, 'val'), transform=val_test_transform)
    test_dataset = ImageFolder(root=os.path.join(DATA_DIR, 'test'), transform=val_test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f'\nTrain samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Test samples: {len(test_dataset)}')
print(f'Number of classes: {len(train_dataset.classes)}')
print(f'Class names: {train_dataset.classes}')


## 5. Inception-V1 (GoogLeNet) Architecture


In [ ]:
class BasicConv2d(nn.Module):
    """Basic convolutional block with Batch Normalization and ReLU"""
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class InceptionBlock(nn.Module):
    """
    Inception block with multiple parallel paths:
    1. 1x1 convolution
    2. 1x1 -> 3x3 convolution
    3. 1x1 -> 5x5 convolution
    4. 3x3 max pooling -> 1x1 convolution
    """
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(InceptionBlock, self).__init__()
        
        # Branch 1: 1x1 convolution
        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)
        
        # Branch 2: 1x1 -> 3x3 convolution
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )
        
        # Branch 3: 1x1 -> 5x5 convolution
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )
        
        # Branch 4: 3x3 max pooling -> 1x1 convolution
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )
    
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        # Concatenate all branches along channel dimension
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


class AuxiliaryClassifier(nn.Module):
    """Auxiliary classifier for training stability (used during training only)"""
    def __init__(self, in_channels, num_classes):
        super(AuxiliaryClassifier, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv = BasicConv2d(in_channels, 128, kernel_size=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


class InceptionV1(nn.Module):
    """
    Inception-V1 (GoogLeNet) architecture from scratch
    
    Architecture:
    - Initial convolutional layers
    - Multiple Inception blocks
    - Auxiliary classifiers (for training stability)
    - Final classifier
    """
    def __init__(self, num_classes=4, aux_logits=True):
        super(InceptionV1, self).__init__()
        self.aux_logits = aux_logits
        
        # Initial convolutional layers
        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Inception blocks (3a, 3b)
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Inception blocks (4a, 4b, 4c, 4d, 4e)
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Inception blocks (5a, 5b)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        
        # Auxiliary classifiers (for training stability)
        if aux_logits:
            self.aux1 = AuxiliaryClassifier(512, num_classes)  # After 4a
            self.aux2 = AuxiliaryClassifier(528, num_classes)  # After 4d
        
        # Final classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        # Initial convolutional layers
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)
        
        # Inception blocks 3a, 3b
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        # Inception blocks 4a, 4b, 4c, 4d, 4e
        x = self.inception4a(x)
        aux1 = None
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        aux2 = None
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)
        
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        # Inception blocks 5a, 5b
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        # Final classifier
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        if self.aux_logits and self.training:
            return x, aux1, aux2
        return x

# Initialize model
model = InceptionV1(num_classes=NUM_CLASSES, aux_logits=True).to(device)

# Print model architecture
print("Inception-V1 (GoogLeNet) Model Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


## 6. Training Configuration


In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer: Adam (as required)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

# Learning rate scheduler: Cosine Annealing (better for training from scratch)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam (lr={LEARNING_RATE}, weight_decay=1e-5)")
print(f"Learning rate scheduler: CosineAnnealingLR (T_max={NUM_EPOCHS}, eta_min=1e-6)")


## 7. Training Loop


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc='Training'):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        
        # Inception outputs main prediction and auxiliary predictions during training
        if model.aux_logits:
            outputs, aux1, aux2 = model(images)
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux1, labels)
            loss3 = criterion(aux2, labels)
            loss = loss1 + 0.3 * loss2 + 0.3 * loss3  # Weighted loss with auxiliary classifiers
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validating'):
            images, labels = images.to(device), labels.to(device)
            
            # During validation, auxiliary classifiers are not used
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc


In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0

print("Starting training...")
print(f"Training for {NUM_EPOCHS} epochs")
print("-" * 60)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print epoch results
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Save best model (based on validation accuracy)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # Disable auxiliary classifiers for inference
        model.aux_logits = False
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'history': history
        }, MODEL_SAVE_PATH)
        model.aux_logits = True  # Re-enable for training
        print(f"Saved best model (Val Acc: {val_acc:.2f}%)")

print("\n" + "=" * 60)
print("Training completed!")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"Model saved to: {MODEL_SAVE_PATH}")


## 8. Training Curves Visualization


In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss curve
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy curve
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('inception_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()


## 9. Load Best Model and Evaluate on Test Set


In [ ]:
# Load best model
checkpoint = torch.load(MODEL_SAVE_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.aux_logits = False  # Disable auxiliary classifiers for inference
print(f"Loaded best model from epoch {checkpoint['epoch']}")
print(f"Best validation accuracy: {checkpoint['val_acc']:.2f}%")

# Evaluate on test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Testing'):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate and print overall accuracy score
accuracy = accuracy_score(all_labels, all_preds)
print(f"\nTest Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")


## 10. Confusion Matrix


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Inception-V1 Model', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('inception_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()


## 11. Classification Report


In [ ]:
# Calculate and print the full classification report
print("Classification Report:")
print("=" * 70)
print(classification_report(all_labels, all_preds, target_names=CLASS_NAMES))
print("=" * 70)


## 12. Summary

### Model Performance Summary:
- **Test Accuracy**: Calculated above
- **Best Validation Accuracy**: Calculated above
- **Model saved to**: `inception_model.pth`

### Architecture Features:
- Full Inception-V1 (GoogLeNet) implementation from scratch
- Inception blocks with parallel convolutions (1x1, 3x3, 5x5, and pooling)
- Batch Normalization after all convolutional layers
- Auxiliary classifiers for training stability
- Adam Optimizer with Cosine Annealing Learning Rate Scheduler

### Files Generated:
1. Trained model: `inception_model.pth`
2. Training curves: `inception_training_curves.png`
3. Confusion matrix: `inception_confusion_matrix.png`

All files are saved in the current directory.
